In this Jupyter notebook, the data resulting from the ALYA simulation will determine the reward for the agents

***

# 0. Introduction 

***

## Importation of useful libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET  # For XML parsing
import pyvista as pv  # For reading and processing mesh data
import pandas as pd  # For data manipulation and DataFrame creation
import plotly.graph_objs as go  # For 3D interactive plot
from plotly.subplots import make_subplots  # For 3D interactive plot
from scipy.ndimage import label  # Function to count the number of clusters
import plotly.express as px  # Package to color each clusters different
from typing import Optional  # For type hinting

print("Libraries imported successfully.")

Libraries imported successfully.


In [2]:
# Change Plotly renderer to browser instead of in notebook
import plotly.io as pio
pio.renderers.default = "browser"

## Constants (Parameters)

In [20]:
# Define the units for the simulation
height = 2.0
Lx = 2.67*height  # Length in the x direction (unitless?)
Ly = height  # Length in the y direction (unitless?)
Lz = 0.8*height  # Length in the z direction (unitless?)

n = 2  # Number of sections in the x direction
m = 2  # Number of sections in the z direction

H = 3  # Sensitivity threshold for Q-event detection


# Path to the directory containing the simulation files
directory_path = (
    "./baseline-coco-100TU/vtk/1end"  # End of the longer run
)

# Path to the file containing pre-calculated averaged data
averaged_data_path = "baseline-coco-100TU/vtk/1end/averaged_data.parquet"  # FOR EXAMPLE

***

# 1. Data preparation 

***

## Loading Data
Upon completion of the ALYA file translation, a VTK folder is generated. This folder contains a .PVD file, which facilitates the opening of the simulation in Paraview and provides timestep information. Additionally, multiple .PVTU files, accompanied by respective .VTU files, store the pertinent data.

In [18]:
def load_data_and_convert_to_dataframe(directory):
    """
    This function loads CFD simulation data from PVTU files and converts them into Pandas DataFrames.
    Each DataFrame is stored along with its respective timestep, facilitating time-series analysis.

    Parameters:
    - directory (str): The path to the directory containing the PVD and PVTU files.

    Returns:
    - data_frames (list of tuples): A list where each element is a tuple containing:
      * A timestep (float)
      * A DataFrame with columns for spatial coordinates (x, y, z) and velocity components (U, V, W)
    """

    # Parse the PVD file to extract mappings of timesteps to their corresponding PVTU files
    pvd_path = os.path.join(directory, "channel.pvd")
    tree = ET.parse(pvd_path)
    root = tree.getroot()
    timestep_file_map = {
        dataset.attrib["file"]: float(dataset.attrib["timestep"])
        for dataset in root.find("Collection")
    }

    # List to store data tuples of timestep and DataFrame
    data_frames = []

    # Process each PVTU file according to its mapped timestep
    for file, timestep in timestep_file_map.items():
        path = os.path.join(directory, file)
        mesh = pv.read(path)  # Read the mesh data from the PVTU file

        # Extract the spatial coordinates and velocity components from the mesh
        points = mesh.points  # x, y, z coordinates
        U, V, W = mesh[
            "VELOC"
        ].T  # Transpose to separate the velocity components (U, V, W)

        # Create a DataFrame with the extracted data
        df = pd.DataFrame(
            {
                "x": points[:, 0],
                "y": points[:, 1],
                "z": points[:, 2],
                "U": U,
                "V": V,
                "W": W,
            }
        )

        # Append the timestep and DataFrame as a tuple to the list
        data_frames.append((timestep, df))
        print(f"Data from {file} at timestep {timestep} loaded into DataFrame.")

    print(f"Total data sets loaded: {len(data_frames)}")
    return data_frames

This predefined function is utilized to extract the data.

In [21]:
# Load the data and store it in `data`
data = load_data_and_convert_to_dataframe(directory_path)

# Print the number of rows for each timestep
# for timestep, df in data:
#     print(f"Number of rows for timestep {timestep}: {len(df)}")
#     print(f"Number of columns for timestep {timestep}: {len(df.columns)}")

# Print the first 20 rows of the first DataFrame to verify the data loading
data[0][1].head(20)

Data from channel_00040000.pvtu at timestep 120.238 loaded into DataFrame.
Total data sets loaded: 1


,x,y,z,U,V,W
0,1.279375,0.005556,0.283333,0.298078,-0.020453,0.041378
1,1.279375,0.005556,0.216667,0.317639,-0.025894,-0.008584
2,1.279375,0.000000,0.283333,0.000000,0.000000,0.000000
3,1.279375,0.000000,0.216667,0.000000,0.000000,0.000000
4,1.056875,0.005556,0.283333,0.220093,-0.014425,0.027735
5,1.056875,0.005556,0.216667,0.256658,-0.027676,0.028771
6,1.056875,0.000000,0.283333,0.000000,0.000000,0.000000
7,1.056875,0.000000,0.216667,0.000000,0.000000,0.000000
8,1.223750,0.005556,0.283333,0.271646,-0.017845,0.020379
9,1.223750,0.005556,0.216667,0.285931,-0.022400,-0.011715


## Normalize the data based on channel dimensions

Scale/Normalize for each timestep, returning in same format as original data:
- data_frames (list of tuples): A list where each element is a tuple containing:
    * A timestep (float)
    * A DataFrame with columns for spatial coordinates (x, y, z) and velocity components (U, V, W)

In [6]:
def normalize_data_frame(
    df: pd.DataFrame, Lx: float, Ly: float, Lz: float
) -> pd.DataFrame:
    """
    Normalizes the spatial coordinates to the range [0, 1] based on their maximum values
    and scales the velocity components by the corresponding channel lengths.

    Parameters:
    - df (pd.DataFrame): Input DataFrame with columns for spatial coordinates (x, y, z) and velocity components (U, V, W).
    - Lx (float): Length in the x direction.
    - Ly (float): Length in the y direction.
    - Lz (float): Length in the z direction.

    Returns:
    - pd.DataFrame: Scaled/Normalized DataFrame.
    """
    # Copy the DataFrame to preserve original data
    df_copy = df.copy()

    # Scale coordinates to 0-1 based on maximum respective coordinate value
    df_copy["x"] = df_copy["x"] / df_copy["x"].max()
    df_copy["y"] = df_copy["y"] / df_copy["y"].max()
    df_copy["z"] = df_copy["z"] / df_copy["z"].max()
    df_copy["U"] = df_copy["U"] / df_copy["x"].max()
    df_copy["V"] = df_copy["V"] / df_copy["y"].max()
    df_copy["W"] = df_copy["W"] / df_copy["z"].max()

    # Normalize to channel dimensions so range is 0-Lx, 0-Ly, 0-Lz
    df_copy["x"] = df_copy["x"] * Lx
    df_copy["y"] = df_copy["y"] * Ly
    df_copy["z"] = df_copy["z"] * Lz
    df_copy["U"] = df_copy["U"] * Lx
    df_copy["V"] = df_copy["V"] * Ly
    df_copy["W"] = df_copy["W"] * Lz

    return df_copy

In [22]:
# Example usage
data_normalized = []

for timestep, df in data:
    # append the normalized data to the list, including timestep information like the original format
    data_normalized.append((timestep, normalize_data_frame(df, Lx, Ly, Lz)))
    print(f"Data normalized for timestep {timestep}.")
    
# Print the first 20 rows of the first DataFrame to verify the data normalization
# data_normalized[0][1].head(20)

Data normalized for timestep 120.238.


***
# 2. Data Processing
***

## Velocity fields definitions

The following table summarizes the key velocity fields used in fluid dynamics, highlighting their formulas and roles:

| Symbol | Description | Definition | Role |
|--------|-------------|------------|------|
| $U(x,y,z,t)$ | Instantaneous velocity field | Represents the velocity at each point $(x, y, z)$ and any time $t$. | Captures detailed fluid dynamics at each point and time. |
| $\overline{U}(y)$ | Averaged velocity field | Averaged over $x$, $z$, and $t$ | Simplifies the 4D field to a function of $y$, smoothing out other variations. |
| $u(x,y,z,t)$ | Velocity fluctuation field | $U(x,y,z,t) - \overline{U}(y)$ | Measures local deviations from the mean, critical for analyzing turbulence. |
| $u'(y)$ | RMS velocity fluctuations | RMS of $u(x,y,z,t)$ across a given dataset | Quantifies the intensity of turbulence or variability along $y$. |

The vertical ($V$ and $v$) and lateral ($W$ and $w$) components of the velocity are defined analogously to the horizontal components ($U$ and $u$).


The upcoming code will compute all these values using the same notation and methods as described.

In [8]:
def process_velocity_data_multiple_timesteps(data, N):
    """
    Processes a list of tuples containing CFD simulation data to calculate averaged and
    fluctuating components of velocity fields over the last N entries. It computes these metrics for
    the horizontal (U), vertical (V), and lateral (W) velocity components.

    Parameters:
    - data (list of tuples): Each tuple contains a timestep and a DataFrame with spatial coordinates (x, y, z)
      and velocity components (U, V, W).
    - N (int): Number of most recent timesteps to include in the averaging process.

    Returns:
    - averaged_data (DataFrame): Contains averaged velocities ($\overline{U}(y)$, $\overline{V}(y)$, $\overline{W}(y)$)
      and rms of velocity fluctuations ($u'(y)$, $v'(y)$, $w'(y)$) as columns, indexed by the y-coordinate.
    - data_process (list of tuples): Each tuple contains a timestep and a DataFrame with original and fluctuating
      velocity components (U, V, W, u, v, w).
    """
    processed_data = []
    recent_data = pd.DataFrame()

    # Aggregate data from the last N timesteps to compute averages and fluctuations
    for timestep, df in data[-N:]:
        df["timestep"] = timestep  # Temporarily add timestep to differentiate data
        recent_data = pd.concat([recent_data, df], ignore_index=True)

    # Calculate mean and standard deviation for u, v, w across the recent data
    averaged_data = (
        recent_data.groupby("y")
        .agg({"U": ["mean", "std"], "V": ["mean", "std"], "W": ["mean", "std"]})
        .rename(columns={"mean": "bar", "std": "prime"}, level=1)
    )
    averaged_data.columns = [
        "U_bar",
        "u_prime",
        "V_bar",
        "v_prime",
        "W_bar",
        "w_prime",
    ]  # Clear column names

    # Process each individual dataset for detailed fluctuation analysis
    for timestep, df in data:
        y_means = averaged_data.loc[df["y"]]
        df_processed = df.copy()
        df_processed["U"] = df["U"]
        df_processed["V"] = df["V"]
        df_processed["W"] = df["W"]
        df_processed["u"] = df["U"] - y_means["U_bar"].values
        df_processed["v"] = df["V"] - y_means["V_bar"].values
        df_processed["w"] = df["W"] - y_means["W_bar"].values

        # Ensure no 'timestep' column remains in the output data
        df_processed.drop(columns="timestep", inplace=True, errors="ignore")
        processed_data.append((timestep, df_processed))

    # Prepare averaged data for output
    averaged_data = averaged_data.reset_index()

    return averaged_data, processed_data

Note : Here we do the average over the all data because it is only the end of the simulation.
But it is important to not take the transitional state for the averaging

In [23]:
N = 1  # Averaging over the last N timesteps
# Note : Here we do the average over the all data because it is only the 1end of the simulation.
# But it is important to not take the transitional state for the averaging
averaged_data, data_process = process_velocity_data_multiple_timesteps(data_normalized, N)

## (FOR UBAR ONLY) Save Averaged Data as .pkl file

In [24]:
def save_dataframe_to_parquet(df: pd.DataFrame, file_path: str):
    """
    Save a DataFrame to a Parquet file.

    Parameters:
    - df (pd.DataFrame): The DataFrame to save.
    - file_path (str): The path to the file where the DataFrame should be saved.
    """
    df.to_parquet(file_path, index=None)

In [ ]:
#averaged_data.head()

In [25]:
# Save the averaged data to a Parquet file for future use
save_dataframe_to_parquet(averaged_data, averaged_data_path)

In [26]:
# test for loading saved parquet file
averaged_data_LOADED = pd.read_parquet(averaged_data_path)

# averaged_data_LOADED.head()

In [ ]:
# Display the first 20 rows of the processed data
# data_process[0][1].head(20)

***
# 3. Q-events dectection 
***

## Explanation

### Objectives and Computational Tools Overview

This section is dedicated to advancing our understanding of turbulent flow dynamics through the identification and analysis of Q-events. The aims of this analysis are threefold:

1. **Q-event Detection**: To accurately identify grid points exhibiting Q-events, which represent significant local interactions in fluid dynamics.
2. **Cluster Analysis**: Upon detection, to quantify these points by clustering, as referenced in the literature under Q-events. This process involves grouping connected Q-events to understand their collective behavior.
3. **Optimization of Sensitivity Threshold (H)**: To determine the value of \(H\) that maximizes the number of Q-event clusters for a given dataset. This parameter tuning is critical for enhancing the detection algorithm's sensitivity and accuracy.

#### Visualization Tools

To facilitate these analyses, several visualization functions have been developed, each corresponding to a specific objective:

- **Q-event Distribution Visualization**: This function allows for the comprehensive visualization of all Q-event points for a selected timestep, providing a macroscopic view of event distributions.
- **Cluster Visualization**: A dedicated function to display Q-events grouped by clusters, aiding in the microscopic examination of the spatial structure and extent of these events.
- **Percolation Diagram**: This function plots the percolation curve, which represents the number of clusters as a function of \(H\). It serves to illustrate how variations in \(H\) influence the clustering behavior, guiding the optimal selection of this threshold.

#### Summary of Functions

The following table encapsulates the functions developed for this analysis, detailing their specific applications and outputs:

| Function Name            | Parameters                                                                                   | Returns                                                                         |
|--------------------------|----------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------|
| `detect_Q_events`        | `processed_data`: List of tuples, `averaged_data`: DataFrame, `H`: float                     | List of tuples, each containing a timestep and a DataFrame with Q event flags  |
| `plot_Q_events_3d`       | `df`: DataFrame containing columns ['x', 'y', 'z', 'Q']                                      | None, plots an interactive 3D scatter plot highlighting Q events                |
| `count_q_clusters`       | `q_event`: DataFrame containing columns ['x', 'y', 'z', 'Q']                                 | Integer, number of distinct clusters in 3D space                                |
| `plot_3d_clusters`       | `q_event`: DataFrame containing columns ['x', 'y', 'z', 'Q']                                 | None, plots an interactive 3D visualization of clusters                         |
| `analyze_q_clusters`     | `processed_data`: List of tuples, `averaged_data`: DataFrame                                 | List of tuples, each containing an H value and the mean number of clusters      |
| `plot_percolation_diagram` | `cluster_analysis_results`: List of tuples                                                   | None, prints and plots the percolation diagram showing the cluster density vs H |

These tools are integral to the methodological exploration of turbulence characteristics and provide crucial insights into the dynamics of turbulent flows.

#### Q-events definition

Q events help identify regions with significant interactions between fluctuating velocities. The detection criterion is as follows:

| Condition | Description | Threshold |
|-----------|-------------|-----------|
| $|u(x,y,z,t) \cdot v(x,y,z,t)| \geq H \cdot u'(y) \cdot v'(y)$ | Detects Q events | The product of fluctuating velocities exceeds $H$ times the product of their RMS values, where $H$ will is determined by maximising the number of Q-events detected. |

## 3a. Detection of Q-events for each time step

In [13]:
def detect_Q_events(processed_data, averaged_data, H):
    """
    Detects Q events in the fluid dynamics data based on the specified condition.

    Parameters:
    - processed_data (list of tuples): Data processed by `process_velocity_data_multiple_timesteps`, containing:
      * A timestep (float)
      * A DataFrame with spatial coordinates (x, y, z) and velocity components U, V, W, u, v, w
    - averaged_data (DataFrame): Data containing the rms values for velocity components u and v for each y coordinate. **(u' and v')**
    - H (float): The sensitivity threshold for identifying Q events.

    Returns:
    - data_frames (list of tuples): Each tuple contains:
      * A timestep (float)
      * A DataFrame with columns ['x', 'y', 'z', 'Q'], where 'Q' is a boolean indicating whether a Q event is detected.
    """
    q_event_data = []

    for timestep, df in processed_data:
        # Fetch the rms values for 'u' and 'v' based on y-coordinate
        rms_values = (
            averaged_data.set_index("y")[["u_prime", "v_prime"]].reindex(df["y"]).values
        )

        # Calculate the product of fluctuating components u and v
        uv_product = np.abs(df["u"] * df["v"])

        # Calculate the threshold product of rms values u' and v'
        threshold = H * rms_values[:, 0] * rms_values[:, 1]

        # Determine where the Q event condition is met
        # q_events = uv_product >= threshold
        q_events = uv_product > threshold  # to avoid detect on 0

        # Create DataFrame with Q event boolean flag
        q_df = pd.DataFrame({"x": df["x"], "y": df["y"], "z": df["z"], "Q": q_events})

        q_event_data.append((timestep, q_df))

    return q_event_data

In [27]:
# Here is an example of the use 
Q_event_frames = detect_Q_events(data_process, averaged_data_LOADED, H)

## 3b. Plot for a particular time step

In [15]:
def plot_Q_events_3d(df):
    """
    Plot an interactive 3D scatter of the points, highlighting the surface where Q is True.

    Parameters:
    - df (DataFrame): DataFrame containing columns ['x', 'y', 'z', 'Q'], where 'Q' is a boolean.
    """
    # Create a figure with plotly
    fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter3d"}]])

    # Filter points where Q is True and False
    df_true = df[df["Q"]]
    df_false = df[~df["Q"]]

    # Add scatter plot for points where Q is False
    trace_false = go.Scatter3d(
        x=df_false["x"],
        y=df_false["z"],
        z=df_false["y"],
        mode="markers",
        marker=dict(
            size=3, color="blue", opacity=0.5  # marker color  # marker opacity
        ),
        name="Q=False",
    )

    # Add scatter plot for points where Q is True
    trace_true = go.Scatter3d(
        x=df_true["x"],
        y=df_true["z"],
        z=df_true["y"],
        mode="markers",
        marker=dict(size=3, color="red", opacity=0.5),  # marker color  # marker opacity
        name="Q=True",
    )

    # Add traces to the figure
    # fig.add_trace(trace_false, row=1, col=1)
    fig.add_trace(trace_true, row=1, col=1)

    # Update layout
    fig.update_layout(
        title="3D Scatter Plot of Q Events",
        scene=dict(
            xaxis_title="X Coordinate",
            yaxis_title="Z Coordinate",
            zaxis_title="Y Coordinate",
        ),
        legend_title="Legend",
        height=700,
        width=800,
    )

    # Show the plot
    fig.show(renderer="browser")

In [28]:
# Timestep of the Q-event to plot
num_timestep = 0
# The Q-event
Q_event = Q_event_frames[num_timestep][1]
# Plot
plot_Q_events_3d(Q_event)

## 3c. Count the local number of Q-events

In [33]:
def calculate_local_Q_ratios(
    df: pd.DataFrame, n: int, m: int, Lx: float, Lz: float, verbose: bool = False
) -> pd.DataFrame:
    """
    Calculate the ratio of Q-events in local volumes for a single timestep.

    Parameters:
    - df (pd.DataFrame): Input DataFrame with columns ['x', 'y', 'z', 'Q'].
    - n (int): Number of sections in the x direction.
    - m (int): Number of sections in the z direction.
    - Lx (float): Length in the x direction.
    - Lz (float): Length in the z direction.

    Returns:
    - pd.DataFrame: DataFrame with columns ['x_index', 'z_index', 'Q_event_count', 'total_points', 'Q_ratio'].
    """
    # Step sizes for local volumes
    step_x = Lx / n
    step_z = Lz / m
    if verbose:
        print(f"step_x = {step_x}")
        print(f"step_z = {step_z}")

    # Initialize list to collect results
    results = []

    # Loop through each local volume
    for i in range(n):
        for j in range(m):
            # Calculate the boundaries of the local volume
            x_min = i * step_x
            x_max = (i + 1) * step_x
            z_min = j * step_z
            z_max = (j + 1) * step_z

            # Filter points within this local volume
            local_points = df[
                (df["x"] >= x_min)
                & (df["x"] < x_max)
                & (df["z"] >= z_min)
                & (df["z"] < z_max)
            ]

            # Calculate counts and Q ratio
            total_points = len(local_points)
            Q_event_count = local_points["Q"].sum()
            Q_ratio = Q_event_count / total_points if total_points > 0 else 0

            # Append results
            results.append(
                {
                    "x_index": i,
                    "z_index": j,
                    "Q_event_count": Q_event_count,
                    "total_points": total_points,
                    "Q_ratio": Q_ratio,
                }
            )

            if verbose:
                print(f"\nlocal volume {i, j}")
                print(f"local volume {i, j} x_min {x_min}")
                print(f"local volume {i, j} x_max {x_max}")
                print(f"local volume {i, j} z_min {z_min}")
                print(f"local volume {i, j} z_max {z_max}\n")
                # print coordinates of every 1000th point in the local volume
                print(
                    f"local volume {i, j} every 1000th filtered point:\n {local_points[::1000]}\n"
                )
                # print every 500th coordinate in filtered points with Q event
                print(
                    f"local volume {i, j} every 500th filtered point with Q event:\n {local_points[local_points['Q']].iloc[::500]}\n"
                )
                print(f"local volume {i, j} total_points {total_points}")
                print(f"local volume {i, j} Q_event_count {Q_event_count}")
                print(f"local volume {i, j} Q_ratio {Q_ratio}\n")

    # Convert results to DataFrame
    result_df = pd.DataFrame(results)

    return result_df


In [35]:
# Example usage
# Process each Q_event_frame individually
verbose = False  # Set to True to display additional information for each local volume
all_results = []

for timestep, df in Q_event_frames:
    result_df = calculate_local_Q_ratios(df, n, m, Lx, Lz, verbose=verbose)
    result_df["timestep"] = timestep  # Add the timestep for context
    all_results.append(result_df)

# Combine all results into a single DataFrame
final_result_df = pd.concat(all_results, ignore_index=True)

# Print the total number of local volumes and total points in each local volume
total_local_volumes = n * m
if verbose:
    print(f"Total number of local volumes: {total_local_volumes}")

# Calculate total points in each local volume for the first timestep
first_timestep_df = all_results[0]
unique_local_volumes = first_timestep_df[["x_index", "z_index"]].drop_duplicates()

for _, volume in unique_local_volumes.iterrows():
    x_index = volume["x_index"]
    z_index = volume["z_index"]
    points_in_volume = first_timestep_df[
        (first_timestep_df["x_index"] == x_index)
        & (first_timestep_df["z_index"] == z_index)
    ]["total_points"].values[0]
    print(f"Local volume ({x_index}, {z_index}) has {points_in_volume} total points.")

Local volume (0, 0) has 115482 total points.
Local volume (0, 1) has 120067 total points.
Local volume (1, 0) has 121624 total points.
Local volume (1, 1) has 124158 total points.


### 3c-1. Additional analysis of local volumes

In [36]:
def plot_local_Q_event_number_bar(
    timestep_data: pd.DataFrame, n: int, m: int, timestep: Optional[float] = None
) -> None:
    """
    Plots the total and Q event points for each local volume/agent.

    Parameters:
    - timestep_data (pd.DataFrame): DataFrame containing local volume data for the specified timestep.
    - n (int): Number of sections in the x direction.
    - m (int): Number of sections in the z direction.
    - timestep (Optional[float]): The specific timestep to plot. If None, it will be ignored in the title.
    """
    # Extract data for the 3D bar chart
    x_indices = timestep_data["x_index"] + 0.5  # Offset by 0.5 to center the bars
    z_indices = timestep_data["z_index"] + 0.5  # Offset by 0.5 to center the bars
    total_points = timestep_data["total_points"]
    Q_event_points = timestep_data["Q_event_count"]

    # Create 3D bars
    bars = []
    # Bars for total points
    for x, z, points in zip(x_indices, z_indices, total_points):
        bars.append(
            go.Scatter3d(
                x=[x - 0.1, x - 0.1],  # Slight offset
                y=[z, z],
                z=[0, points],
                mode="lines",
                line=dict(color="blue", width=10),
                name="Total Points",
            )
        )

    # Bars for Q-event points
    for x, z, points in zip(x_indices, z_indices, Q_event_points):
        bars.append(
            go.Scatter3d(
                x=[x + 0.1, x + 0.1],  # Slight offset
                y=[z, z],
                z=[0, points],
                mode="lines",
                line=dict(color="red", width=10),
                name="Q-event Points",
            )
        )

    title = f"Total Points and Q-event Points"
    if timestep is not None:
        title += f" for Timestep {timestep}"

    fig = go.Figure(data=bars)

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(
                title="X Index",
                range=[0, n],
                tickvals=list(range(n + 1)),  # Set gridlines to integer values
                showgrid=True,
                gridcolor="lightgrey",
            ),
            yaxis=dict(
                title="Z Index",
                range=[0, m],
                tickvals=list(range(m + 1)),  # Set gridlines to integer values
                showgrid=True,
                gridcolor="lightgrey",
            ),
            zaxis=dict(title="Points", showgrid=True, gridcolor="lightgrey"),
        ),
        showlegend=False,  # Show legend separately
        height=800,  # Set the height of the figure
    )

    fig.show()

In [37]:
# Example usage
verbose_local_volume = (
    False  # Set to True to display additional information for each local volume
)

# Process each Q_event_frame individually and collect results in a dictionary
results_by_timestep = {}

for timestep, df in Q_event_frames:
    result_df = calculate_local_Q_ratios(df, n, m, Lx, Lz, verbose=verbose_local_volume)
    results_by_timestep[timestep] = result_df

# Select the specific timestep you want to plot
timestep_to_plot_index = 0  # Index of the timestep to plot
timestep_to_plot = list(results_by_timestep.keys())[timestep_to_plot_index]
# timestep_to_plot = 895.2  # Specify the timestep you want to plot
timestep_data = results_by_timestep[timestep_to_plot]

if verbose_local_volume:
    print(f"n = {n}, m = {m}, Lx = {Lx}, Ly = {Ly}, Lz = {Lz}")

    # Print the total number of local volumes
    total_local_volumes = n * m
    print(f"Total number of local volumes: {total_local_volumes}\n")

    # Additional print statements for the selected timestep
    min_total_points = timestep_data["total_points"].min()
    max_total_points = timestep_data["total_points"].max()
    avg_total_points = timestep_data["total_points"].mean()
    std_dev_total_points = timestep_data["total_points"].std()
    percent_deviation_total_points = (std_dev_total_points / avg_total_points) * 100

    min_Q_event_points = timestep_data["Q_event_count"].min()
    max_Q_event_points = timestep_data["Q_event_count"].max()
    avg_Q_event_points = timestep_data["Q_event_count"].mean()
    std_dev_Q_event_points = timestep_data["Q_event_count"].std()
    percent_deviation_Q_event_points = (
        std_dev_Q_event_points / avg_Q_event_points
    ) * 100

    print(f"List of all timesteps: {list(results_by_timestep.keys())}\n")
    print(f"Selected timestep: {timestep_to_plot}\n")
    print(f"Minimum number of total points per local volume: {min_total_points}")
    print(f"Maximum number of total points per local volume: {max_total_points}")
    print(f"Average number of total points per local volume: {avg_total_points}")
    print(
        f"Standard deviation of total points per local volume: {std_dev_total_points}"
    )
    print(
        f"Percent deviation of total points per local volume: {percent_deviation_total_points:.2f}%\n"
    )

    print(f"Minimum number of Q-event points per local volume: {min_Q_event_points}")
    print(f"Maximum number of Q-event points per local volume: {max_Q_event_points}")
    print(f"Average number of Q-event points per local volume: {avg_Q_event_points}")
    print(
        f"Standard deviation of Q-event points per local volume: {std_dev_Q_event_points}"
    )
    print(
        f"Percent deviation of Q-event points per local volume: {percent_deviation_Q_event_points:.2f}%\n"
    )

plot_local_Q_event_number_bar(timestep_data, n, m)


In [38]:
def plot_local_Q_events_3d(
    df: pd.DataFrame,
    n: int,
    m: int,
    Lx: float,
    Ly: float,
    Lz: float,
    local_x_index: int,
    local_z_index: int,
    switch_y_z: bool = True,
) -> None:
    """
    Plot an interactive 3D scatter of the points, highlighting the Q-event points in a specified local volume.

    Parameters:
    - df (pd.DataFrame): DataFrame containing columns ['x', 'y', 'z', 'Q'], where 'Q' is a boolean.
    - n (int): Number of sections in the x direction.
    - m (int): Number of sections in the z direction.
    - Lx (float): Length in the x direction.
    - Ly (float): Length in the y direction.
    - Lz (float): Length in the z direction.
    - local_x_index (int): Index of the local volume in the x direction.
    - local_z_index (int): Index of the local volume in the z direction.
    """
    # Calculate the boundaries of the specified local volume
    step_x = Lx / n
    step_z = Lz / m
    local_x_min = local_x_index * step_x
    local_x_max = (local_x_index + 1) * step_x
    local_z_min = local_z_index * step_z
    local_z_max = (local_z_index + 1) * step_z

    # Filter points within this local volume
    local_points = df[
        (df["x"] >= local_x_min)
        & (df["x"] <= local_x_max)
        & (df["z"] >= local_z_min)
        & (df["z"] <= local_z_max)
    ]

    # Create a figure with plotly
    fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter3d"}]])

    # Filter points where Q is True within the global volume
    global_points_true = df[df["Q"]]

    # Filter points where Q is True and False within the local volume
    local_points_true = local_points[local_points["Q"]]
    
    if switch_y_z:
        # Add scatter plot for points where Q is True in the global volume
        trace_global_true = go.Scatter3d(
            x=global_points_true["x"],
            y=global_points_true[
                "z"
            ],  # switch y and z points to get better default orientation
            z=global_points_true["y"],
            mode="markers",
            marker=dict(size=3, color="red", opacity=0.5),  # marker color  # marker opacity
            name="Q=True (Global)",
        )

        # Add scatter plot for points where Q is True in the local volume
        trace_local_true = go.Scatter3d(
            x=local_points_true["x"],
            y=local_points_true[
                "z"
            ],  # switch y and z points to get better default orientation
            z=local_points_true["y"],
            mode="markers",
            marker=dict(
                size=5, color="yellow", opacity=0.8  # marker color  # marker opacity
            ),
            name="Q=True (Local)",
        )
        
    else:
        # Add scatter plot for points where Q is True in the global volume
        trace_global_true = go.Scatter3d(
            x=global_points_true["x"],
            y=global_points_true["y"],
            z=global_points_true["z"],
            mode="markers",
            marker=dict(size=3, color="red", opacity=0.5),  # marker color  # marker opacity
            name="Q=True (Global)",
        )

        # Add scatter plot for points where Q is True in the local volume
        trace_local_true = go.Scatter3d(
            x=local_points_true["x"],
            y=local_points_true["y"],
            z=local_points_true["z"],
            mode="markers",
            marker=dict(
                size=5, color="yellow", opacity=0.8  # marker color  # marker opacity
            ),
            name="Q=True (Local)",
        )

    # Add traces to the figure
    fig.add_trace(trace_global_true, row=1, col=1)
    fig.add_trace(trace_local_true, row=1, col=1)

    # Create a wireframe of the local volume
    if switch_y_z:
        volume_outline = [
            # Bottom face
            [local_x_min, local_x_max, local_x_max, local_x_min, local_x_min, local_x_min],
            [local_z_min, local_z_min, local_z_max, local_z_max, local_z_min, local_z_min],
            [0, 0, 0, 0, 0, 0],  # y values for bottom face
            # Top face
            [local_x_min, local_x_max, local_x_max, local_x_min, local_x_min, local_x_min],
            [local_z_min, local_z_min, local_z_max, local_z_max, local_z_min, local_z_min],
            [Ly, Ly, Ly, Ly, Ly, Ly],  # y values for top face
            # Vertical lines
            [
                local_x_min,
                local_x_min,
                local_x_max,
                local_x_max,
                local_x_max,
                local_x_max,
                local_x_min,
                local_x_min,
            ],
            [
                local_z_min,
                local_z_min,
                local_z_min,
                local_z_min,
                local_z_max,
                local_z_max,
                local_z_max,
                local_z_max,
            ],
            [0, Ly, 0, Ly, 0, Ly, 0, Ly],  # y values for vertical lines
        ]
    else:
        volume_outline = [
            # Bottom face
            [local_x_min, local_x_max, local_x_max, local_x_min, local_x_min, local_x_min],
            [0, 0, 0, 0, 0, 0],  # y values for bottom face
            [local_z_min, local_z_min, local_z_max, local_z_max, local_z_min, local_z_min],
            # Top face
            [local_x_min, local_x_max, local_x_max, local_x_min, local_x_min, local_x_min],
            [Ly, Ly, Ly, Ly, Ly, Ly],  # y values for top face
            [local_z_min, local_z_min, local_z_max, local_z_max, local_z_min, local_z_min],
            # Vertical lines
            [
                local_x_min,
                local_x_min,
                local_x_max,
                local_x_max,
                local_x_max,
                local_x_max,
                local_x_min,
                local_x_min,
            ],
            [0, Ly, 0, Ly, 0, Ly, 0, Ly],  # y values for vertical lines
            [
                local_z_min,
                local_z_min,
                local_z_min,
                local_z_min,
                local_z_max,
                local_z_max,
                local_z_max,
                local_z_max,
            ],
        ]

    trace_outline = go.Scatter3d(
        x=sum(volume_outline[0::3], []),  # x values
        y=sum(volume_outline[1::3], []),  # z values (y axis in plot)
        z=sum(volume_outline[2::3], []),  # y values
        mode="lines",
        line=dict(color="blue", width=2, dash="dot"),
        name="Local Volume Outline",
    )

    fig.add_trace(trace_outline, row=1, col=1)

    # Update layout
    if switch_y_z:
        fig.update_layout(
            title=f"3D Scatter Plot of Q Events in Local Volume ({local_x_index}, {local_z_index})",
            scene=dict(
                xaxis=dict(
                    title="X Index",
                    range=[0, Lx],  # Normalized range [0, 1]
                    tickvals=[i * (Lx / n) for i in range(n + 1)],  # Normalized positions
                    ticktext=[str(i) for i in range(n + 1)],  # Custom labels as integers
                    showgrid=True,
                    gridcolor="lightgrey",
                ),
                yaxis=dict(
                    title="Z Index",
                    range=[0, Lz],  # Normalized range [0, 1]
                    tickvals=[i* (Lz / m) for i in range(m + 1)],  # Normalized positions
                    ticktext=[str(i) for i in range(m + 1)],  # Custom labels as integers
                    showgrid=True,
                    gridcolor="lightgrey",
                ),
                zaxis=dict(
                    title="Y Coordinate",
                    range=[0, Ly],
                    showgrid=True,
                    gridcolor="lightgrey",
                ),
            ),
            legend_title="Legend",
            height=800,
            width=1000
        )
    else:
        fig.update_layout(
            title=f"3D Scatter Plot of Q Events in Local Volume ({local_x_index}, {local_z_index})",
            scene=dict(
                xaxis=dict(
                    title="X Index",
                    range=[0, Lx],  # Normalized range [0, 1]
                    tickvals=[i * (Lx / n) for i in range(n + 1)],  # Normalized positions
                    ticktext=[str(i) for i in range(n + 1)],  # Custom labels as integers
                    showgrid=True,
                    gridcolor="lightgrey",
                ),
                yaxis=dict(
                    title="Y Coordinate",
                    range=[0, Ly],
                    showgrid=True,
                    gridcolor="lightgrey",
                ),
                zaxis=dict(
                    title="Z Index",
                    range=[0, Lz],  # Normalized range [0, 1]
                    tickvals=[i * (Lz / m) for i in range(m + 1)],  # Normalized positions
                    ticktext=[str(i) for i in range(m + 1)],  # Custom labels as integers
                    showgrid=True,
                    gridcolor="lightgrey",
                ),
            )
        )

    # Show the plot
    fig.show(renderer="browser")

### 3c-2. Visualization of local volume Q-events

In [39]:
local_x_index = 0  # Index of the local volume in the x direction
local_z_index = 1  # Index of the local volume in the z direction

if local_x_index > n or local_z_index > m:
    raise ValueError(
        "Local volume indices must be within the specified number of sections."
    )

# Timestep of the Q-event to plot
num_timestep = 0
# The Q-event
Q_event_local_plotting = Q_event_frames[num_timestep][1]

# Highlight the Q-events in the specified local volume
plot_local_Q_events_3d(
    Q_event_local_plotting, n, m, Lx, Ly, Lz, local_x_index, local_z_index, switch_y_z=False
)

# 4. Reward Function

## Explanation

The goal of the reward is to quantify the quality of the local flow field based on the detected Q-events. The reward function should reflect the turbulence intensity and the presence of significant local interactions. The following criteria are currently considered in the reward function:

1. **Q-event Density**: The density of Q-events in the local volume, indicating the intensity of local interactions, reflected by `Q_ratio`.

Possible future reward options include:

2. **Cluster Size**: The size of the largest Q-event cluster, reflecting the spatial extent of significant interactions.
3. **Cluster Density**: The density of Q-event clusters, providing insights into the distribution of local interactions.
4. **Cluster Connectivity**: The connectivity of Q-event clusters, capturing the coherence of local interactions.
5. **Q-event Persistence**: The duration of Q-events, indicating the temporal stability of local interactions.
6. **Q-event Intensity**: The intensity of Q-events, reflecting the strength of local interactions.
7. **Q-event Diversity**: The variety of Q-event types, capturing different interaction patterns.
8. **Q-event Dynamics**: The evolution of Q-events over time, providing insights into local flow dynamics.
9. **Q-event Interactions**: The interactions between Q-events, highlighting complex flow structures.
10. **Q-event Anomalies**: The presence of unusual or rare Q-events, indicating unique flow characteristics.

The current reward function is as follows:

`R` = 1 - `Q_ratio`

## Calculating Reward Based on Q-Ratio

In [40]:
def calculate_reward(df: pd.DataFrame, n: int, m: int) -> float:
    """
    Calculate the reward based on the Q ratio in the local volume.

    Parameters:
    - df (pd.DataFrame): DataFrame with columns ['x_index', 'z_index', 'Q_ratio'].

    Returns:
    - float: The calculated reward value.
    """
    # Get specified Q_ratio value based on index
    q_ratio = df[(df["x_index"] == n) & (df["z_index"] == m)]["Q_ratio"].values[0]
    # Calculate the reward based on the Q ratio for the specified index
    reward = 1 - q_ratio

    return reward

In [41]:
# Example usage
local_x_index = 0  # Index of the local volume in the x direction
local_z_index = 0  # Index of the local volume in the z direction

# Calculate the reward for the specified local volume
reward_value = calculate_reward(timestep_data, local_x_index, local_z_index)

# Create a DataFrame with reward value appended to `timestep_data`
reward_df = timestep_data.copy()
for i in range(n):
    for j in range(m):
        reward_df.loc[
            (reward_df["x_index"] == i) & (reward_df["z_index"] == j), "reward"
        ] = calculate_reward(timestep_data, i, j)

print(reward_df)

   x_index  z_index  Q_event_count  total_points   Q_ratio    reward
0        0        0           1812        115482  0.015691  0.984309
1        0        1           3306        120067  0.027535  0.972465
2        1        0           3891        121624  0.031992  0.968008
3        1        1           3670        124158  0.029559  0.970441
